The `environment.yml` file is at the bottom of this notebook. You can use it to recreate my environment with `conda`. 

In [1]:
from dotenv import load_dotenv
import math
from matplotlib import pyplot as plt
import os
from web3 import Web3

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Load environment variables from .env file
load_dotenv()
MAINNET_RPC_URL = os.environ.get('MAINNET_RPC_URL')
web3 = Web3(Web3.HTTPProvider(MAINNET_RPC_URL))

In UniswapV3 current price between the `token0` and `token1` is represented by the `sqrtPriceX96`. It can be read from the `UniswapV3Pool#slot0`. 

As an example let's look at the Ethereum Mainnet USDC / ETH 0.05% pool.

In [3]:
# Ethereum Mainnet
usdc_eth_pool_address = '0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640'

To interact with existing contracts using `web3.py` we need contract's abi and address

In [4]:
pool_abi = '[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"int24","name":"tickLower","type":"int24"},{"indexed":true,"internalType":"int24","name":"tickUpper","type":"int24"},{"indexed":false,"internalType":"uint128","name":"amount","type":"uint128"},{"indexed":false,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1","type":"uint256"}],"name":"Burn","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":false,"internalType":"address","name":"recipient","type":"address"},{"indexed":true,"internalType":"int24","name":"tickLower","type":"int24"},{"indexed":true,"internalType":"int24","name":"tickUpper","type":"int24"},{"indexed":false,"internalType":"uint128","name":"amount0","type":"uint128"},{"indexed":false,"internalType":"uint128","name":"amount1","type":"uint128"}],"name":"Collect","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":true,"internalType":"address","name":"recipient","type":"address"},{"indexed":false,"internalType":"uint128","name":"amount0","type":"uint128"},{"indexed":false,"internalType":"uint128","name":"amount1","type":"uint128"}],"name":"CollectProtocol","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":true,"internalType":"address","name":"recipient","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"paid0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"paid1","type":"uint256"}],"name":"Flash","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint16","name":"observationCardinalityNextOld","type":"uint16"},{"indexed":false,"internalType":"uint16","name":"observationCardinalityNextNew","type":"uint16"}],"name":"IncreaseObservationCardinalityNext","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint160","name":"sqrtPriceX96","type":"uint160"},{"indexed":false,"internalType":"int24","name":"tick","type":"int24"}],"name":"Initialize","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"sender","type":"address"},{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"int24","name":"tickLower","type":"int24"},{"indexed":true,"internalType":"int24","name":"tickUpper","type":"int24"},{"indexed":false,"internalType":"uint128","name":"amount","type":"uint128"},{"indexed":false,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1","type":"uint256"}],"name":"Mint","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint8","name":"feeProtocol0Old","type":"uint8"},{"indexed":false,"internalType":"uint8","name":"feeProtocol1Old","type":"uint8"},{"indexed":false,"internalType":"uint8","name":"feeProtocol0New","type":"uint8"},{"indexed":false,"internalType":"uint8","name":"feeProtocol1New","type":"uint8"}],"name":"SetFeeProtocol","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":true,"internalType":"address","name":"recipient","type":"address"},{"indexed":false,"internalType":"int256","name":"amount0","type":"int256"},{"indexed":false,"internalType":"int256","name":"amount1","type":"int256"},{"indexed":false,"internalType":"uint160","name":"sqrtPriceX96","type":"uint160"},{"indexed":false,"internalType":"uint128","name":"liquidity","type":"uint128"},{"indexed":false,"internalType":"int24","name":"tick","type":"int24"}],"name":"Swap","type":"event"},{"inputs":[{"internalType":"int24","name":"tickLower","type":"int24"},{"internalType":"int24","name":"tickUpper","type":"int24"},{"internalType":"uint128","name":"amount","type":"uint128"}],"name":"burn","outputs":[{"internalType":"uint256","name":"amount0","type":"uint256"},{"internalType":"uint256","name":"amount1","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"int24","name":"tickLower","type":"int24"},{"internalType":"int24","name":"tickUpper","type":"int24"},{"internalType":"uint128","name":"amount0Requested","type":"uint128"},{"internalType":"uint128","name":"amount1Requested","type":"uint128"}],"name":"collect","outputs":[{"internalType":"uint128","name":"amount0","type":"uint128"},{"internalType":"uint128","name":"amount1","type":"uint128"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint128","name":"amount0Requested","type":"uint128"},{"internalType":"uint128","name":"amount1Requested","type":"uint128"}],"name":"collectProtocol","outputs":[{"internalType":"uint128","name":"amount0","type":"uint128"},{"internalType":"uint128","name":"amount1","type":"uint128"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"factory","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"fee","outputs":[{"internalType":"uint24","name":"","type":"uint24"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"feeGrowthGlobal0X128","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"feeGrowthGlobal1X128","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount0","type":"uint256"},{"internalType":"uint256","name":"amount1","type":"uint256"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"flash","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint16","name":"observationCardinalityNext","type":"uint16"}],"name":"increaseObservationCardinalityNext","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint160","name":"sqrtPriceX96","type":"uint160"}],"name":"initialize","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"liquidity","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"maxLiquidityPerTick","outputs":[{"internalType":"uint128","name":"","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"int24","name":"tickLower","type":"int24"},{"internalType":"int24","name":"tickUpper","type":"int24"},{"internalType":"uint128","name":"amount","type":"uint128"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"mint","outputs":[{"internalType":"uint256","name":"amount0","type":"uint256"},{"internalType":"uint256","name":"amount1","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"observations","outputs":[{"internalType":"uint32","name":"blockTimestamp","type":"uint32"},{"internalType":"int56","name":"tickCumulative","type":"int56"},{"internalType":"uint160","name":"secondsPerLiquidityCumulativeX128","type":"uint160"},{"internalType":"bool","name":"initialized","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint32[]","name":"secondsAgos","type":"uint32[]"}],"name":"observe","outputs":[{"internalType":"int56[]","name":"tickCumulatives","type":"int56[]"},{"internalType":"uint160[]","name":"secondsPerLiquidityCumulativeX128s","type":"uint160[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"name":"positions","outputs":[{"internalType":"uint128","name":"liquidity","type":"uint128"},{"internalType":"uint256","name":"feeGrowthInside0LastX128","type":"uint256"},{"internalType":"uint256","name":"feeGrowthInside1LastX128","type":"uint256"},{"internalType":"uint128","name":"tokensOwed0","type":"uint128"},{"internalType":"uint128","name":"tokensOwed1","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"protocolFees","outputs":[{"internalType":"uint128","name":"token0","type":"uint128"},{"internalType":"uint128","name":"token1","type":"uint128"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint8","name":"feeProtocol0","type":"uint8"},{"internalType":"uint8","name":"feeProtocol1","type":"uint8"}],"name":"setFeeProtocol","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"slot0","outputs":[{"internalType":"uint160","name":"sqrtPriceX96","type":"uint160"},{"internalType":"int24","name":"tick","type":"int24"},{"internalType":"uint16","name":"observationIndex","type":"uint16"},{"internalType":"uint16","name":"observationCardinality","type":"uint16"},{"internalType":"uint16","name":"observationCardinalityNext","type":"uint16"},{"internalType":"uint8","name":"feeProtocol","type":"uint8"},{"internalType":"bool","name":"unlocked","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"int24","name":"tickLower","type":"int24"},{"internalType":"int24","name":"tickUpper","type":"int24"}],"name":"snapshotCumulativesInside","outputs":[{"internalType":"int56","name":"tickCumulativeInside","type":"int56"},{"internalType":"uint160","name":"secondsPerLiquidityInsideX128","type":"uint160"},{"internalType":"uint32","name":"secondsInside","type":"uint32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"bool","name":"zeroForOne","type":"bool"},{"internalType":"int256","name":"amountSpecified","type":"int256"},{"internalType":"uint160","name":"sqrtPriceLimitX96","type":"uint160"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"swap","outputs":[{"internalType":"int256","name":"amount0","type":"int256"},{"internalType":"int256","name":"amount1","type":"int256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"int16","name":"","type":"int16"}],"name":"tickBitmap","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"tickSpacing","outputs":[{"internalType":"int24","name":"","type":"int24"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"int24","name":"","type":"int24"}],"name":"ticks","outputs":[{"internalType":"uint128","name":"liquidityGross","type":"uint128"},{"internalType":"int128","name":"liquidityNet","type":"int128"},{"internalType":"uint256","name":"feeGrowthOutside0X128","type":"uint256"},{"internalType":"uint256","name":"feeGrowthOutside1X128","type":"uint256"},{"internalType":"int56","name":"tickCumulativeOutside","type":"int56"},{"internalType":"uint160","name":"secondsPerLiquidityOutsideX128","type":"uint160"},{"internalType":"uint32","name":"secondsOutside","type":"uint32"},{"internalType":"bool","name":"initialized","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"token0","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"token1","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"}]'

Once we have this data, we can create a contract instance.

In [5]:
usdc_eth_pool = web3.eth.contract(address=usdc_eth_pool_address, abi=pool_abi)

Given the instance, it is possible to read contract state.

In [6]:
# Slot0 contains: [sqrtPriceX96 uint160, tick int24, observationIndex uint16, 
# observationCardinality uint16, observationCardinalityNext uint16, feeProtocol uint8, unlocked bool]
slot0 = usdc_eth_pool.functions.slot0().call()
print(slot0)

[1851538011312559681199299953156713, 201193, 255, 722, 722, 0, True]


`X96` means that the value is multiplied by $2^{96}$

In our example the `sqrtPriceX96` is simply `sqrtPrice` multiplied by $2^{96}$

Let's calculate the ETH price that is showing on the frontend.

$price =\ ?$

1. Divide both sides by $2^{96}$

$$sqrtPrice = {sqrtPriceX96\over{2^{96}}}$$

2. Raise to the power of 2 to remove the square root

$$price = {sqrtPriceX96\over{2^{96}}}^2$$

In [7]:
# Let's take the sqrtPriceX96 from slot0 and plug it in the equation
sqrtPriceX96 = slot0[0]

# I will hardcode the price here, so that it shows on GitHub
# you don't have to do this, if you are following in your own notebook

sqrtPriceX96 = 1851131613155175002238687231876895 # Price as of 7th August, 2023

price = (sqrtPriceX96 / (2 ** 96)) ** 2
print(price)

545902933.1808959


The result we got is not something that you would expect, right?

There are 2 things that needs to be kept in mind:

1. The `token0` in this pool is USDC and `token1` is WETH. The price we got is the exchange rate $token1\over{token0}$, so how much WETH can be bought for 1 USDC. Exchanges show the inverse.
2. Tokens have different decimals, we have to account for that. 

Let's adjust the decimals first. 

To do so we need to divide the price by: $${token1_{decimals}}\over{token2_{decimals}}$$

In [8]:
price = price / ((10 ** 18) / (10 ** 6))
print(price)

0.000545902933180896


Let's invert the price now.

In [9]:
price = 1 / price
print(price)

1831.8274902337441



`environment.yml`:
```
name: base
channels:
  - defaults
dependencies:
  - _ipyw_jlab_nb_ext_conf=0.1.0=py39hecd8cb5_1
  - alabaster=0.7.12=pyhd3eb1b0_0
  - anaconda=2022.10=py39_0
  - anaconda-client=1.11.0=py39hecd8cb5_0
  - anaconda-navigator=2.3.2=py39hecd8cb5_0
  - anaconda-project=0.11.1=py39hecd8cb5_0
  - anyio=3.5.0=py39hecd8cb5_0
  - appdirs=1.4.4=pyhd3eb1b0_0
  - applaunchservices=0.3.0=py39hecd8cb5_0
  - appnope=0.1.2=py39hecd8cb5_1001
  - appscript=1.1.2=py39h9ed2024_0
  - argon2-cffi=21.3.0=pyhd3eb1b0_0
  - argon2-cffi-bindings=21.2.0=py39hca72f7f_0
  - arrow=1.2.2=pyhd3eb1b0_0
  - astroid=2.11.7=py39hecd8cb5_0
  - astropy=5.1=py39h67323c0_0
  - atomicwrites=1.4.0=py_0
  - attrs=21.4.0=pyhd3eb1b0_0
  - automat=20.2.0=py_0
  - autopep8=1.6.0=pyhd3eb1b0_1
  - babel=2.9.1=pyhd3eb1b0_0
  - backcall=0.2.0=pyhd3eb1b0_0
  - backports=1.1=pyhd3eb1b0_0
  - backports.functools_lru_cache=1.6.4=pyhd3eb1b0_0
  - backports.tempfile=1.0=pyhd3eb1b0_1
  - backports.weakref=1.0.post1=py_1
  - bcrypt=3.2.0=py39hca72f7f_1
  - beautifulsoup4=4.11.1=py39hecd8cb5_0
  - binaryornot=0.4.4=pyhd3eb1b0_1
  - bitarray=2.5.1=py39hca72f7f_0
  - bkcharts=0.2=py39hecd8cb5_1
  - black=22.6.0=py39hecd8cb5_0
  - blas=1.0=mkl
  - bleach=4.1.0=pyhd3eb1b0_0
  - blosc=1.21.0=h8346a28_1
  - bokeh=2.4.3=py39hecd8cb5_0
  - boto3=1.24.28=py39hecd8cb5_0
  - botocore=1.27.28=py39hecd8cb5_0
  - bottleneck=1.3.5=py39h67323c0_0
  - brotli=1.0.9=hca72f7f_7
  - brotli-bin=1.0.9=hca72f7f_7
  - brotlipy=0.7.0=py39h9ed2024_1003
  - brunsli=0.1=h23ab428_0
  - bzip2=1.0.8=h1de35cc_0
  - c-ares=1.18.1=hca72f7f_0
  - ca-certificates=2022.07.19=hecd8cb5_0
  - cctools=949.0.1=h9abeeb2_25
  - cctools_osx-64=949.0.1=hc7db93f_25
  - certifi=2022.9.24=py39hecd8cb5_0
  - cffi=1.15.1=py39hc55c11b_0
  - cfitsio=3.470=hbd21bf8_7
  - chardet=4.0.0=py39hecd8cb5_1003
  - charls=2.2.0=h23ab428_0
  - charset-normalizer=2.0.4=pyhd3eb1b0_0
  - click=8.0.4=py39hecd8cb5_0
  - cloudpickle=2.0.0=pyhd3eb1b0_0
  - clyent=1.2.2=py39hecd8cb5_1
  - colorama=0.4.5=py39hecd8cb5_0
  - colorcet=3.0.0=py39hecd8cb5_0
  - conda=23.1.0=py39hecd8cb5_0
  - conda-build=3.22.0=py39hecd8cb5_0
  - conda-content-trust=0.1.3=py39hecd8cb5_0
  - conda-env=2.6.0=1
  - conda-pack=0.6.0=pyhd3eb1b0_0
  - conda-package-handling=2.2.0=py39hecd8cb5_0
  - conda-package-streaming=0.9.0=py39hecd8cb5_0
  - conda-repo-cli=1.0.41=py39hecd8cb5_0
  - conda-token=0.4.0=pyhd3eb1b0_0
  - conda-verify=3.4.2=py_1
  - constantly=15.1.0=pyh2b92418_0
  - cookiecutter=1.7.3=pyhd3eb1b0_0
  - cryptography=37.0.1=py39hf6deb26_0
  - cssselect=1.1.0=pyhd3eb1b0_0
  - curl=7.84.0=hca72f7f_0
  - cycler=0.11.0=pyhd3eb1b0_0
  - cython=0.29.32=py39he9d5cce_0
  - cytoolz=0.11.0=py39h9ed2024_0
  - daal4py=2021.6.0=py39h01d92e1_1
  - dal=2021.6.0=haf03e11_929
  - dask=2022.7.0=py39hecd8cb5_0
  - dask-core=2022.7.0=py39hecd8cb5_0
  - dataclasses=0.8=pyh6d0b6a4_7
  - datashader=0.14.1=py39hecd8cb5_0
  - datashape=0.5.4=py39hecd8cb5_1
  - debugpy=1.5.1=py39he9d5cce_0
  - decorator=5.1.1=pyhd3eb1b0_0
  - defusedxml=0.7.1=pyhd3eb1b0_0
  - diff-match-patch=20200713=pyhd3eb1b0_0
  - dill=0.3.4=pyhd3eb1b0_0
  - distributed=2022.7.0=py39hecd8cb5_0
  - docutils=0.18.1=py39hecd8cb5_3
  - entrypoints=0.4=py39hecd8cb5_0
  - et_xmlfile=1.1.0=py39hecd8cb5_0
  - fftw=3.3.9=h9ed2024_1
  - filelock=3.6.0=pyhd3eb1b0_0
  - flake8=4.0.1=pyhd3eb1b0_1
  - flask=1.1.2=pyhd3eb1b0_0
  - fonttools=4.25.0=pyhd3eb1b0_0
  - freetype=2.11.0=hd8bbffd_0
  - fsspec=2022.7.1=py39hecd8cb5_0
  - future=0.18.2=py39hecd8cb5_1
  - gensim=4.1.2=py39he9d5cce_0
  - gettext=0.21.0=h7535e17_0
  - giflib=5.2.1=haf1e3a3_0
  - glib=2.69.1=h8346a28_1
  - glob2=0.7=pyhd3eb1b0_0
  - gmp=6.2.1=he9d5cce_3
  - gmpy2=2.1.2=py39hd5de756_0
  - greenlet=1.1.1=py39h23ab428_0
  - gst-plugins-base=1.14.0=h4180768_2
  - gstreamer=1.14.0=h0fc69c2_2
  - h5py=3.7.0=py39h4a1dd59_0
  - hdf5=1.10.6=h10fe05b_1
  - heapdict=1.0.1=pyhd3eb1b0_0
  - holoviews=1.15.0=py39hecd8cb5_0
  - hvplot=0.8.0=py39hecd8cb5_0
  - hyperlink=21.0.0=pyhd3eb1b0_0
  - icu=58.2=h0a44026_3
  - idna=3.3=pyhd3eb1b0_0
  - imagecodecs=2021.8.26=py39h0f85e6e_1
  - imageio=2.19.3=py39hecd8cb5_0
  - imagesize=1.4.1=py39hecd8cb5_0
  - importlib-metadata=4.11.3=py39hecd8cb5_0
  - importlib_metadata=4.11.3=hd3eb1b0_0
  - incremental=21.3.0=pyhd3eb1b0_0
  - inflection=0.5.1=py39hecd8cb5_0
  - iniconfig=1.1.1=pyhd3eb1b0_0
  - intake=0.6.5=pyhd3eb1b0_0
  - intel-openmp=2021.4.0=hecd8cb5_3538
  - intervaltree=3.1.0=pyhd3eb1b0_0
  - ipykernel=6.15.2=py39hecd8cb5_0
  - ipython=7.31.1=py39hecd8cb5_1
  - ipython_genutils=0.2.0=pyhd3eb1b0_1
  - ipywidgets=7.6.5=pyhd3eb1b0_1
  - isort=5.9.3=pyhd3eb1b0_0
  - itemadapter=0.3.0=pyhd3eb1b0_0
  - itemloaders=1.0.4=pyhd3eb1b0_1
  - itsdangerous=2.0.1=pyhd3eb1b0_0
  - jdcal=1.4.1=pyhd3eb1b0_0
  - jedi=0.18.1=py39hecd8cb5_1
  - jellyfish=0.9.0=py39hca72f7f_0
  - jinja2=2.11.3=pyhd3eb1b0_0
  - jinja2-time=0.2.0=pyhd3eb1b0_3
  - jmespath=0.10.0=pyhd3eb1b0_0
  - joblib=1.1.0=pyhd3eb1b0_0
  - jpeg=9e=hca72f7f_0
  - jq=1.6=h9ed2024_1000
  - json5=0.9.6=pyhd3eb1b0_0
  - jsonschema=4.16.0=py39hecd8cb5_0
  - jupyter=1.0.0=py39hecd8cb5_8
  - jupyter_client=7.3.4=py39hecd8cb5_0
  - jupyter_console=6.4.3=pyhd3eb1b0_0
  - jupyter_core=4.11.1=py39hecd8cb5_0
  - jupyter_server=1.18.1=py39hecd8cb5_0
  - jupyterlab=3.4.4=py39hecd8cb5_0
  - jupyterlab_pygments=0.1.2=py_0
  - jupyterlab_server=2.10.3=pyhd3eb1b0_1
  - jupyterlab_widgets=1.0.0=pyhd3eb1b0_1
  - jxrlib=1.1=haf1e3a3_2
  - keyring=23.4.0=py39hecd8cb5_0
  - kiwisolver=1.4.2=py39he9d5cce_0
  - krb5=1.19.2=hcd88c3b_0
  - lazy-object-proxy=1.6.0=py39h9ed2024_0
  - lcms2=2.12=hf1fd2bf_0
  - ld64=530=h20443b4_25
  - ld64_osx-64=530=h70f3046_25
  - ldid=2.1.2=h2d21305_2
  - lerc=3.0=he9d5cce_0
  - libaec=1.0.4=hb1e8313_1
  - libarchive=3.6.1=he336d3b_0
  - libbrotlicommon=1.0.9=hca72f7f_7
  - libbrotlidec=1.0.9=hca72f7f_7
  - libbrotlienc=1.0.9=hca72f7f_7
  - libclang=12.0.0=default_hbc2896b_2
  - libcurl=7.84.0=h6dfd666_0
  - libcxx=14.0.6=h9765a3e_0
  - libdeflate=1.8=h9ed2024_5
  - libedit=3.1.20210910=hca72f7f_0
  - libev=4.33=h9ed2024_1
  - libffi=3.3=hb1e8313_2
  - libgfortran=5.0.0=11_2_0_h246ff09_26
  - libgfortran5=11.2.0=h246ff09_26
  - libiconv=1.16=hca72f7f_2
  - libidn2=2.3.2=h9ed2024_0
  - liblief=0.11.5=he9d5cce_1
  - libllvm11=11.1.0=h46f1229_5
  - libllvm12=12.0.0=h9b2ccf5_3
  - libllvm14=14.0.6=he552d86_0
  - libnghttp2=1.46.0=ha29bfda_0
  - libpng=1.6.37=ha441bb4_0
  - libpq=12.9=h1c9f633_3
  - libsodium=1.0.18=h1de35cc_0
  - libspatialindex=1.9.3=h23ab428_0
  - libssh2=1.10.0=h0a4fc7d_0
  - libtiff=4.4.0=h2ef1027_0
  - libunistring=0.9.10=h9ed2024_0
  - libwebp=1.2.2=h56c3ce4_0
  - libwebp-base=1.2.2=hca72f7f_0
  - libxml2=2.9.14=hbf8cd5e_0
  - libxslt=1.1.35=h5b33f42_0
  - libzopfli=1.0.3=hb1e8313_0
  - llvm-openmp=14.0.6=h0dcd299_0
  - llvmlite=0.38.0=py39h8346a28_0
  - locket=1.0.0=py39hecd8cb5_0
  - lxml=4.9.1=py39h65b224f_0
  - lz4=3.1.3=py39h9ed2024_0
  - lz4-c=1.9.3=h23ab428_1
  - lzo=2.10=haf1e3a3_2
  - markdown=3.3.4=py39hecd8cb5_0
  - markupsafe=2.0.1=py39h9ed2024_0
  - matplotlib=3.5.2=py39hecd8cb5_0
  - matplotlib-base=3.5.2=py39hfb0c5b7_0
  - matplotlib-inline=0.1.6=py39hecd8cb5_0
  - mccabe=0.7.0=pyhd3eb1b0_0
  - mistune=0.8.4=py39h9ed2024_1000
  - mkl=2021.4.0=hecd8cb5_637
  - mkl-service=2.4.0=py39h9ed2024_0
  - mkl_fft=1.3.1=py39h4ab4a9b_0
  - mkl_random=1.2.2=py39hb2f4e1b_0
  - mock=4.0.3=pyhd3eb1b0_0
  - mpc=1.1.0=h6ef4df4_1
  - mpfr=4.0.2=h9066e36_1
  - mpi=1.0=mpich
  - mpich=3.3.2=external_0
  - mpmath=1.2.1=py39hecd8cb5_0
  - msgpack-python=1.0.3=py39haf03e11_0
  - multipledispatch=0.6.0=py39hecd8cb5_0
  - munkres=1.1.4=py_0
  - mypy_extensions=0.4.3=py39hecd8cb5_1
  - navigator-updater=0.3.0=py39hecd8cb5_0
  - nbclassic=0.3.5=pyhd3eb1b0_0
  - nbclient=0.5.13=py39hecd8cb5_0
  - nbconvert=6.4.4=py39hecd8cb5_0
  - nbformat=5.5.0=py39hecd8cb5_0
  - ncurses=6.3=hca72f7f_3
  - nest-asyncio=1.5.5=py39hecd8cb5_0
  - networkx=2.8.4=py39hecd8cb5_0
  - nltk=3.7=pyhd3eb1b0_0
  - nose=1.3.7=pyhd3eb1b0_1008
  - notebook=6.4.12=py39hecd8cb5_0
  - nspr=4.33=he9d5cce_0
  - nss=3.74=h47edf6a_0
  - numba=0.55.1=py39hae1ba45_0
  - numexpr=2.8.3=py39h2e5f0a9_0
  - numpy=1.21.5=py39h2e5f0a9_3
  - numpy-base=1.21.5=py39h3b1a694_3
  - numpydoc=1.4.0=py39hecd8cb5_0
  - olefile=0.46=pyhd3eb1b0_0
  - oniguruma=6.9.7.1=h9ed2024_0
  - openjpeg=2.4.0=h66ea3da_0
  - openpyxl=3.0.10=py39hca72f7f_0
  - openssl=1.1.1q=hca72f7f_0
  - packaging=21.3=pyhd3eb1b0_0
  - pandas=1.4.4=py39he9d5cce_0
  - pandocfilters=1.5.0=pyhd3eb1b0_0
  - panel=0.13.1=py39hecd8cb5_0
  - param=1.12.0=pyhd3eb1b0_0
  - parsel=1.6.0=py39hecd8cb5_0
  - parso=0.8.3=pyhd3eb1b0_0
  - partd=1.2.0=pyhd3eb1b0_1
  - patch=2.7.6=h1de35cc_1001
  - pathlib=1.0.1=pyhd3eb1b0_1
  - pathspec=0.9.0=py39hecd8cb5_0
  - patsy=0.5.2=py39hecd8cb5_1
  - pcre=8.45=h23ab428_0
  - pep8=1.7.1=py39hecd8cb5_1
  - pexpect=4.8.0=pyhd3eb1b0_3
  - pickleshare=0.7.5=pyhd3eb1b0_1003
  - pillow=9.2.0=py39hde71d04_1
  - pip=22.2.2=py39hecd8cb5_0
  - pkginfo=1.8.2=pyhd3eb1b0_0
  - platformdirs=2.5.2=py39hecd8cb5_0
  - plotly=5.9.0=py39hecd8cb5_0
  - pluggy=1.0.0=py39hecd8cb5_1
  - ply=3.11=py39hecd8cb5_0
  - poyo=0.5.0=pyhd3eb1b0_0
  - prometheus_client=0.14.1=py39hecd8cb5_0
  - prompt-toolkit=3.0.20=pyhd3eb1b0_0
  - prompt_toolkit=3.0.20=hd3eb1b0_0
  - protego=0.1.16=py_0
  - psutil=5.9.0=py39hca72f7f_0
  - ptyprocess=0.7.0=pyhd3eb1b0_2
  - py=1.11.0=pyhd3eb1b0_0
  - py-lief=0.11.5=py39he9d5cce_1
  - pyasn1=0.4.8=pyhd3eb1b0_0
  - pyasn1-modules=0.2.8=py_0
  - pycodestyle=2.8.0=pyhd3eb1b0_0
  - pycosat=0.6.3=py39h9ed2024_0
  - pycparser=2.21=pyhd3eb1b0_0
  - pyct=0.4.8=py39hecd8cb5_1
  - pycurl=7.45.1=py39h0a4fc7d_0
  - pydispatcher=2.0.5=py39hecd8cb5_2
  - pydocstyle=6.1.1=pyhd3eb1b0_0
  - pyerfa=2.0.0=py39h9ed2024_0
  - pyflakes=2.4.0=pyhd3eb1b0_0
  - pygments=2.11.2=pyhd3eb1b0_0
  - pyhamcrest=2.0.2=pyhd3eb1b0_2
  - pyjwt=2.4.0=py39hecd8cb5_0
  - pylint=2.14.5=py39hecd8cb5_0
  - pyls-spyder=0.4.0=pyhd3eb1b0_0
  - pyobjc-core=8.5=py39hc55c11b_0
  - pyobjc-framework-cocoa=8.5=py39hca72f7f_0
  - pyobjc-framework-coreservices=8.5=py39hca72f7f_0
  - pyobjc-framework-fsevents=8.5=py39hecd8cb5_0
  - pyodbc=4.0.34=py39he9d5cce_0
  - pyopenssl=22.0.0=pyhd3eb1b0_0
  - pyparsing=3.0.9=py39hecd8cb5_0
  - pyqt=5.15.7=py39he9d5cce_0
  - pyqt5-sip=12.11.0=py39he9d5cce_0
  - pyqtwebengine=5.15.7=py39he9d5cce_0
  - pyrsistent=0.18.0=py39hca72f7f_0
  - pysocks=1.7.1=py39hecd8cb5_0
  - pytables=3.6.1=py39h648f197_1
  - pytest=7.1.2=py39hecd8cb5_0
  - python=3.9.13=hdfd78df_1
  - python-dateutil=2.8.2=pyhd3eb1b0_0
  - python-fastjsonschema=2.16.2=py39hecd8cb5_0
  - python-libarchive-c=2.9=pyhd3eb1b0_1
  - python-lsp-black=1.2.1=py39hecd8cb5_0
  - python-lsp-jsonrpc=1.0.0=pyhd3eb1b0_0
  - python-lsp-server=1.5.0=py39hecd8cb5_0
  - python-slugify=5.0.2=pyhd3eb1b0_0
  - python-snappy=0.6.0=py39h23ab428_3
  - python.app=3=py39hca72f7f_0
  - pytz=2022.1=py39hecd8cb5_0
  - pyviz_comms=2.0.2=pyhd3eb1b0_0
  - pywavelets=1.3.0=py39hca72f7f_0
  - pyyaml=6.0=py39hca72f7f_1
  - pyzmq=23.2.0=py39he9d5cce_0
  - qdarkstyle=3.0.2=pyhd3eb1b0_0
  - qstylizer=0.1.10=pyhd3eb1b0_0
  - qt=5.15.9=hecd8cb5_0
  - qt-main=5.15.2=h719ae48_7
  - qt-webengine=5.15.9=h90a370e_4
  - qtawesome=1.0.3=pyhd3eb1b0_0
  - qtconsole=5.3.2=py39hecd8cb5_0
  - qtpy=2.2.0=py39hecd8cb5_0
  - qtwebkit=5.212=h24dc246_4
  - queuelib=1.5.0=py39hecd8cb5_0
  - readline=8.1.2=hca72f7f_1
  - regex=2022.7.9=py39hca72f7f_0
  - requests=2.28.1=py39hecd8cb5_0
  - requests-file=1.5.1=pyhd3eb1b0_0
  - ripgrep=13.0.0=hc2228c6_0
  - rope=0.22.0=pyhd3eb1b0_0
  - rtree=0.9.7=py39hecd8cb5_1
  - ruamel.yaml=0.17.21=py39hca72f7f_0
  - ruamel.yaml.clib=0.2.6=py39hca72f7f_1
  - ruamel_yaml=0.15.100=py39h9ed2024_0
  - s3transfer=0.6.0=py39hecd8cb5_0
  - scikit-image=0.19.2=py39hae1ba45_0
  - scikit-learn=1.0.2=py39hae1ba45_1
  - scikit-learn-intelex=2021.6.0=py39hecd8cb5_0
  - scipy=1.9.1=py39h3d31255_0
  - scrapy=2.6.2=py39hecd8cb5_0
  - seaborn=0.11.2=pyhd3eb1b0_0
  - send2trash=1.8.0=pyhd3eb1b0_1
  - service_identity=18.1.0=pyhd3eb1b0_1
  - setuptools=63.4.1=py39hecd8cb5_0
  - sip=6.6.2=py39he9d5cce_0
  - six=1.16.0=pyhd3eb1b0_1
  - smart_open=5.2.1=py39hecd8cb5_0
  - snappy=1.1.9=he9d5cce_0
  - sniffio=1.2.0=py39hecd8cb5_1
  - snowballstemmer=2.2.0=pyhd3eb1b0_0
  - sortedcollections=2.1.0=pyhd3eb1b0_0
  - sortedcontainers=2.4.0=pyhd3eb1b0_0
  - soupsieve=2.3.1=pyhd3eb1b0_0
  - sphinx=5.0.2=py39hecd8cb5_0
  - sphinxcontrib-applehelp=1.0.2=pyhd3eb1b0_0
  - sphinxcontrib-devhelp=1.0.2=pyhd3eb1b0_0
  - sphinxcontrib-htmlhelp=2.0.0=pyhd3eb1b0_0
  - sphinxcontrib-jsmath=1.0.1=pyhd3eb1b0_0
  - sphinxcontrib-qthelp=1.0.3=pyhd3eb1b0_0
  - sphinxcontrib-serializinghtml=1.1.5=pyhd3eb1b0_0
  - spyder=5.3.3=py39hecd8cb5_0
  - spyder-kernels=2.3.3=py39hecd8cb5_0
  - sqlalchemy=1.4.39=py39hca72f7f_0
  - sqlite=3.39.3=h707629a_0
  - statsmodels=0.13.2=py39hca72f7f_0
  - sympy=1.10.1=py39hecd8cb5_0
  - tabulate=0.8.10=py39hecd8cb5_0
  - tapi=1000.10.8=ha1b3eb9_0
  - tbb=2021.6.0=haf03e11_0
  - tbb4py=2021.6.0=py39haf03e11_0
  - tblib=1.7.0=pyhd3eb1b0_0
  - tenacity=8.0.1=py39hecd8cb5_1
  - terminado=0.13.1=py39hecd8cb5_0
  - testpath=0.6.0=py39hecd8cb5_0
  - text-unidecode=1.3=pyhd3eb1b0_0
  - textdistance=4.2.1=pyhd3eb1b0_0
  - threadpoolctl=2.2.0=pyh0d69192_0
  - three-merge=0.1.1=pyhd3eb1b0_0
  - tifffile=2021.7.2=pyhd3eb1b0_2
  - tinycss=0.4=pyhd3eb1b0_1002
  - tk=8.6.12=h5d9f67b_0
  - tldextract=3.2.0=pyhd3eb1b0_0
  - toml=0.10.2=pyhd3eb1b0_0
  - tomli=2.0.1=py39hecd8cb5_0
  - tomlkit=0.11.1=py39hecd8cb5_0
  - toolz=0.11.2=pyhd3eb1b0_0
  - tornado=6.1=py39h9ed2024_0
  - tqdm=4.64.1=py39hecd8cb5_0
  - traitlets=5.1.1=pyhd3eb1b0_0
  - twisted=22.2.0=py39hca72f7f_1
  - typing-extensions=4.3.0=py39hecd8cb5_0
  - typing_extensions=4.3.0=py39hecd8cb5_0
  - tzdata=2022c=h04d1e81_0
  - ujson=5.4.0=py39he9d5cce_0
  - unidecode=1.2.0=pyhd3eb1b0_0
  - unixodbc=2.3.11=hb456775_0
  - urllib3=1.26.11=py39hecd8cb5_0
  - w3lib=1.21.0=pyhd3eb1b0_0
  - watchdog=2.1.6=py39h999c104_0
  - wcwidth=0.2.5=pyhd3eb1b0_0
  - webencodings=0.5.1=py39hecd8cb5_1
  - websocket-client=0.58.0=py39hecd8cb5_4
  - werkzeug=2.0.3=pyhd3eb1b0_0
  - wget=1.21.3=h6dfd666_0
  - whatthepatch=1.0.2=py39hecd8cb5_0
  - wheel=0.37.1=pyhd3eb1b0_0
  - widgetsnbextension=3.5.2=py39hecd8cb5_0
  - wrapt=1.14.1=py39hca72f7f_0
  - wurlitzer=3.0.2=py39hecd8cb5_0
  - xarray=0.20.1=pyhd3eb1b0_1
  - xlrd=2.0.1=pyhd3eb1b0_0
  - xlsxwriter=3.0.3=pyhd3eb1b0_0
  - xlwings=0.27.15=py39hecd8cb5_0
  - xz=5.2.6=hca72f7f_0
  - yaml=0.2.5=haf1e3a3_0
  - yapf=0.31.0=pyhd3eb1b0_0
  - zeromq=4.3.4=h23ab428_0
  - zfp=0.5.5=he9d5cce_6
  - zict=2.1.0=py39hecd8cb5_0
  - zipp=3.8.0=py39hecd8cb5_0
  - zlib=1.2.12=h4dc903c_3
  - zope=1.0=py39hecd8cb5_1
  - zope.interface=5.4.0=py39h9ed2024_0
  - zstandard=0.19.0=py39h6c40b1e_0
  - zstd=1.5.2=hcb37349_0
  - pip:
      - aiohttp==3.8.5
      - aiosignal==1.3.1
      - async-timeout==4.0.2
      - eth-abi==4.1.0
      - eth-account==0.9.0
      - eth-hash==0.5.2
      - eth-keyfile==0.6.1
      - eth-keys==0.4.0
      - eth-rlp==0.3.0
      - eth-typing==3.4.0
      - eth-utils==2.2.0
      - frozenlist==1.4.0
      - hexbytes==0.3.1
      - lru-dict==1.2.0
      - multidict==6.0.4
      - parsimonious==0.9.0
      - protobuf==4.23.4
      - pycryptodome==3.18.0
      - pyunormalize==15.0.0
      - rlp==3.0.0
      - web3==6.8.0
      - websockets==11.0.3
      - yarl==1.9.2
prefix: /Users/kamilchmielewski/opt/anaconda3
```